# Car-following model (Base code)

This notebook was written by Evangelos Paschalidis (evangelos.paschalidis@epfl.ch) for the Decision-aid methodologies in transportation course at EPFL (http://edu.epfl.ch/coursebook/en/decision-aid-methodologies-in-transportation-CIVIL-557). 

Please contact before distributing or reusing the material below.

## Overview

This notebook covers the estimation of a linear regression model in python with maximum likelihhod estimation:

* Load necessary packages
* Define variables and parameters to estimate
* Model specification
* Model output

Have a go at working through the notebook. To run a code cell, just click on it (to see a green box around it) and then press the **Run** button at the top! 

Some cells have blank lines for you to complete. There is always a comment telling you what to do!

You can also add a new cell by pressing the **+** button at the top of the page.

## Load packages

Before we estimate the model let's load some packages that we are going to need. When importing a package, it is common to rename it using an abbreviation.

In [1]:
import pandas as pd # for data frame processing
import numpy as np # for some statistical procedures
from scipy.optimize import minimize # optimisation routine for parameter estimation
from scipy.stats import norm # normal distribution density function
import numdifftools as nd # we use this to calculate t-ratios and p-values
import csv # we need this to store our parameters as csv

# We've got some rookies in the mix.
from scipy.special import roots_legendre # we use this to generate nodes for numerical integration
from scipy.stats import qmc # we use this to generate nodes for numerical integration

### Let's give a name to our model

In [2]:
model_name = 'car_following_model_importance_sampling_helly' # Name we want to give to our model (this is used when saving the output)

### Panel structure
We need to define whether our data is panel (i.e. multiple observations per individual) or not

In [3]:
panel = 1 # switch to 1 if data is panel (any other value if not panel)

### Define if we use mixing

In [4]:
mixing = 1 # switch to 1 if we apply mixing (any other value if no mixing applied)

## Load the data

Now it is time to load the data. We can do that using the piece of code below.

**Important!** Make sure the data is in the same folder with the notebook file

In [5]:
# Command to load the data
data = pd.read_table('I80_data0.txt')

# Number of observations (we need this to caclulate goodness-of-fit indices)
Nobs = data.shape[0]

## Print the data

Let's quickly print the data. Simply type *data* in the field below

(If we had opened our data set with a different name e.g. *database*

In [6]:
# Type "data" in this field (without the quotation) and run the cell (Shift + return)
data

,ID,Time,Position,Length,Width,Type,Speed,Acceleration,Lane,Leader,...,lead04,lane01,lane02,lane03,lane04,tasks,task_index,timelag,dd,idd
0,4,26,83.033604,4.084802,-9999.99,2,6.847780,-2.266411,5,21,...,1,1,1,1,1,28,5,25,1,4
1,4,27,87.759061,4.084802,-9999.99,2,3.295830,-1.349331,5,21,...,1,1,1,1,1,28,6,26,1,4
2,4,28,90.960723,4.084802,-9999.99,2,2.820909,-2.040139,5,21,...,1,1,1,1,1,28,7,27,1,4
3,4,29,92.047244,4.084802,-9999.99,2,0.276981,0.345661,5,21,...,1,1,1,1,1,28,8,28,1,4
4,4,30,92.601824,4.084802,-9999.99,2,0.634731,0.365870,5,21,...,1,1,1,1,1,28,9,29,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18421,3340,110,397.345232,4.359100,-9999.99,2,10.053292,-0.356570,2,117,...,1,1,1,1,1,55,51,109,1,3340
18422,3340,111,406.291822,4.359100,-9999.99,2,8.161121,-0.587411,2,117,...,1,1,1,1,1,55,52,110,1,3340
18423,3340,112,414.387694,4.359100,-9999.99,2,8.106680,0.110621,2,117,...,1,1,1,1,1,55,53,111,1,3340
18424,3340,113,422.633135,4.359100,-9999.99,2,8.298771,-0.182639,2,117,...,1,1,1,1,1,55,54,112,1,3340


## Print the variable names (columns)

We can also print the variable names only using the piece of code below

* This is useful during model specification to easily access the column names

In [7]:
print(data.columns)

Index(['ID', 'Time', 'Position', 'Length', 'Width', 'Type', 'Speed',
       'Acceleration', 'Lane', 'Leader', 'Follower', 'Space_headway',
       'Time_headway', 'Acceleration_lead', 'Speed_lead', 'Position_lead',
       'Density', 'distanceToEnd', 'lane_change', 'lead_change', 'lag_leader0',
       'lag_leader1', 'lag_leader2', 'lag_leader3', 'lag_leader4', 'lag_lane0',
       'lag_lane1', 'lag_lane2', 'lag_lane3', 'lag_lane4', 'lag_acceleration0',
       'lag_acceleration1', 'lag_acceleration2', 'lag_acceleration3',
       'lag_acceleration4', 'lag_acceleration_lead0', 'lag_acceleration_lead1',
       'lag_acceleration_lead2', 'lag_acceleration_lead3',
       'lag_acceleration_lead4', 'lag_speed0', 'lag_speed1', 'lag_speed2',
       'lag_speed3', 'lag_speed4', 'lag_speed_lead0', 'lag_speed_lead1',
       'lag_speed_lead2', 'lag_speed_lead3', 'lag_speed_lead4',
       'lag_position_lead0', 'lag_position_lead1', 'lag_position_lead2',
       'lag_position_lead3', 'lag_position_lead4', '

## Data processing

* It may be the case that our data do not include all variables required for modelling.
* In this case we must generate the additional variables that we need

### Variable creation

* Let's create a "running task" variable, else a counter for the observations of each driver

In [8]:
data['running_task'] = data.groupby(['ID']).cumcount()+1 # counter of observations per individual

## Variable definition

We need to define the variables (as numpy arrays) that we will use in our model.

* The arrays can have any name but it is more convenient to use the same name as in the data set.

In [9]:
# Example:
# Variable_name = np.array(data['Variable_name']).reshape(-1, 1)

running_task = np.array(data['running_task']).reshape(-1, 1)
Acceleration = np.array(data['Acceleration']).reshape(-1, 1)
Speed = np.array(data['Speed']).reshape(-1, 1)
Space_headway = np.array(data['Space_headway']).reshape(-1, 1)
Speed_lead = np.array(data['Speed_lead']).reshape(-1, 1)
Acceleration_lead = np.array(data['Acceleration_lead']).reshape(-1, 1)

#### Define the "lagged" speed variables

In [10]:
# Lagged speed values #
lag_speed1 = np.array(data['lag_speed1']).reshape(-1, 1)
lag_speed2 = np.array(data['lag_speed2']).reshape(-1, 1)
lag_speed3 = np.array(data['lag_speed3']).reshape(-1, 1)

# Lagged speed lead values #
lag_speed_lead1 = np.array(data['lag_speed_lead1']).reshape(-1, 1)
lag_speed_lead2 = np.array(data['lag_speed_lead2']).reshape(-1, 1)
lag_speed_lead3 = np.array(data['lag_speed_lead3']).reshape(-1, 1)

# Lagged acceleration values #
lag_acceleration1 = np.array(data['lag_acceleration1']).reshape(-1, 1)
lag_acceleration2 = np.array(data['lag_acceleration2']).reshape(-1, 1)
lag_acceleration3 = np.array(data['lag_acceleration3']).reshape(-1, 1)

# Lagged speed lead values #
lag_acceleration_lead1 = np.array(data['lag_acceleration_lead1']).reshape(-1, 1)
lag_acceleration_lead2 = np.array(data['lag_acceleration_lead2']).reshape(-1, 1)
lag_acceleration_lead3 = np.array(data['lag_acceleration_lead3']).reshape(-1, 1)

#### Define the ID variable

In [11]:
ID = np.array(data['ID']) # ID does not need to be reshaped

## Model specification

We now need to create a function that includes our model.

* Python functions are defined as: def *function_name*(parameters):
* We end a function as: return *value_to_return*

In the current implementation we specify two different functions as:
* *function 1* calculates the log likelihood per observations
* *function 2* calculates the sum of log likelihood taking as input the result from *function 1*

*We define two different functions to be more flexible in the post estimation processing later in the code*

We use some python (numpy) functions such '*exp*' or '*log*'. To execute these in the current example, we need to also call numpy; hence, we have *np.exp()* and *np.log()*.

### Define parameters and starting values

Ultimately, we want to estimate the value of some parameters that maximise the likelihood of our observations of the dependent variable.

Before starting the estimation process, we need to define some starting values for the parameters to be estimated.

* The starting values are usually zeroes
* When a parameter is included as a denominator, the starting value cannot be 0 for computational reasons.
* However, since we estimate the log of sigma, our starting value can be zero since exp(sigma) can never be absolute zero.

In [12]:
betas_start = {
               "tau_mean":0, # Acceleration constant parameter
               "sigma_tau":0, # Deceleration constant parameter
               "alpha1":0, # Acceleration constant parameter
               "alpha2":0, # Deceleration constant parameter
               "beta1":0,  # Speed (acceleration regime) parameter
               "beta2":0,  # Speed (deceleration regime) parameter
               "sigma":0  # Std deviation (deceleration regime) parameter
                }

### Load old parameter estimates results

Sometimes, we want to use results from old models as starting values.
* To do that, we will load the iteration file from a previous estimation
* Please note that only values of parameters with same names with our current model will be copied

In [13]:
# ### Activate this cell to load old results ###

# # Open old iteration file
# betas_old = pd.read_csv('model_name_iterations.csv')

# # Keep last row
# betas_old = betas_old.tail(1)

# # Convert to dictionary
# betas_old = betas_old.to_dict(orient='records')[0]

# # Copy values from old to start for keys that are common to both dictionaries
# betas_start = {k: betas_old[k] for k in betas_start.keys() & betas_old.keys()}

# # Delete old estimates
# del[betas_old]

### Generation of random draws
In this piece of code below, we generate the random draws to be used as reaction time.

Two pieces of code are provided one for numerical integration and one for simulation

**Only one of the two blocks can be activated at a time!!**

In [14]:
# First let's set the number of draws
Ndraws = 50

# And then let's set the reaction time range
tau_max = 3 # Max bound of reaction time
tau_min = 0 # Min bound of reaction time

#### Method 1: Code for numerical integration
With numerical integration we directly get reaction time values in the (0, 3) range (or any other range we set)

In [15]:
# # This piece of code is generating the nodes and weights using the number of nodes we set just above
# roots, weights = roots_legendre(Ndraws) # generate (-1, 1) nodes

# tau_max = 3 # Max bound of reaction time
# tau_min = 0 # Min bound of reaction time
# tau = 0.5*(tau_max-tau_min)*roots+0.5*(tau_max+tau_min) # convert nodes (-1, 1) to reaction time values (0, 3)

# # Some data manipulation to correct the reaction time matix dimensions
# tau = np.transpose(tau)
# tau = np.tile(tau, (len(data), 1))

#### Method 2: Code for simulation
The simulation code will generate uniform random numbers (0, 1).

These numbers must be then used in the model function (Function 1) to convert them to reaction time values using truncated distribution simulation.

In [16]:
# This piece of code is generating Halton draws using the number of nodes we set just above
nIndiv = len(set(ID))
draws = ['tau_draws']
nDim = len(draws)
tasks = (pd.DataFrame(ID).value_counts(sort = False)) # observations per ID

sampler = qmc.Halton(d=nDim, scramble=False)
sample = pd.DataFrame(sampler.random(n=Ndraws*nIndiv+Ndraws)) # +Ndraws
sample = sample[(Ndraws-1):(Ndraws*nIndiv+Ndraws-1)]

cols = len(sample.columns)

for i in range(cols):
    # print(i)
    sample1 = np.array(sample.loc[:,i])
    sample1=pd.DataFrame(np.reshape(sample1,(nIndiv,Ndraws)))
    sample_rep =sample1.loc[sample1.index.repeat(tasks)]
    globals()[draws[i]] = sample_rep

#### Generate importance sampling distribution (for importance sampling only)

In [17]:
# logmean and sigma for the importance sampling distribution
tau_mean0 = 0
sigma_tau0 = 0

# Reaction time distribution (truncated log normal)    
tau = np.exp(norm.ppf(
        # norm.cdf(np.log(tau_min),tau_mean,np.exp(sigma_tau))+
        tau_draws*(norm.cdf(np.log(tau_max),tau_mean0,np.exp(sigma_tau0))) # -norm.cdf(np.log(tau_min),tau_mean,np.exp(sigma_tau))
        )*np.exp(sigma_tau0)+tau_mean0)

#### Calculate lagged speed values based on the reaction time values

This step is **Only** relevant for numerical integration.

If simulation is used then the block of code below must be pasted inside Function 1 and not before it.

In [18]:
#################################################################################################
# Here we calculate the lagged relative speed given the reaction time values we generated above #
#################################################################################################

# The way this is done is by implementing linear interpolation between the observations around each reaction time value
# We simply assume that "acceleration = (speed1 - speed0) / (time1 - time0)"
# We take the observation closest to the reaction time value

# We calculate lag speed
lag_speed = ((np.intc(tau)==0) * ((np.intc(tau)+1-tau<0.5)*(lag_speed1+lag_acceleration1*(np.intc(tau)+1-tau))+
                                    (np.intc(tau)+1-tau>=0.5)*(Speed-Acceleration*(tau-np.intc(tau))))+
             (np.intc(tau)==1) * ((np.intc(tau)+1-tau<0.5)*(lag_speed2+lag_acceleration2*(np.intc(tau)+1-tau))+
                            (np.intc(tau)+1-tau>=0.5)*(lag_speed1-lag_acceleration1*(tau-np.intc(tau))))+
             (np.intc(tau)==2) * ((np.intc(tau)+1-tau<0.5)*(lag_speed3+lag_acceleration3*(np.intc(tau)+1-tau))+
                            (np.intc(tau)+1-tau>=0.5)*(lag_speed2-lag_acceleration2*(tau-np.intc(tau)))))

# We correct lag speed if the result is negative
lag_speed = (lag_speed*(lag_speed>=0))+((np.round(tau)==0)*Speed+(np.round(tau)==1)*lag_speed1+(np.round(tau)==2)*lag_speed2+(np.round(tau)==3)*lag_speed3)*(lag_speed<0)

# We calculate lag speed of leader
lag_speed_lead = ((np.intc(tau)==0) * ((np.intc(tau)+1-tau<0.5)*(lag_speed_lead1+lag_acceleration_lead1*(np.intc(tau)+1-tau))+(np.intc(tau)+1-tau>=0.5)*(Speed_lead-Acceleration_lead*(tau-np.intc(tau))))+
                  (np.intc(tau)==1) * ((np.intc(tau)+1-tau<0.5)*(lag_speed_lead2+lag_acceleration_lead2*(np.intc(tau)+1-tau))+(np.intc(tau)+1-tau>=0.5)*(lag_speed_lead1-lag_acceleration_lead1*(tau-np.intc(tau))))+
                  (np.intc(tau)==2) * ((np.intc(tau)+1-tau<0.5)*(lag_speed_lead3+lag_acceleration_lead3*(np.intc(tau)+1-tau))+(np.intc(tau)+1-tau>=0.5)*(lag_speed_lead2-lag_acceleration_lead2*(tau-np.intc(tau)))))

# We correct the lag speed of leader if the result is negative
lag_speed_lead = (lag_speed_lead*(lag_speed_lead>=0))+((np.round(tau)==0)*Speed_lead+(np.round(tau)==1)*lag_speed_lead1+(np.round(tau)==2)*lag_speed_lead2+(np.round(tau)==3)*lag_speed_lead3)*(lag_speed_lead<0)


# We calculate lagged relative speed as the difference
Lagged_relative_speed = lag_speed_lead-lag_speed # Lagged relative speed per reaction time value

### Function 1: log likelihood (LL)
This function calculates the log likelihood per observation

In [19]:
def LL(betas): # betas is a vector with the parameters we want to estimate
   
    # First let's define the parameters to be estimated.
    # The parameter names are imported directly from 'beta_start' that we defined earlier
    
    for pn in range(len(betas_start.values())):
        globals()[np.array(list(betas_start.keys()))[pn]] = betas[pn]

    ############################################################################################################    
    ############################################################################################################    
    # Calculation of reaction time values
    # This part of the code must ONLY be activated if simulation is used
    
    #     tau = np.exp(norm.ppf(
    #        tau_draws*(norm.cdf(np.log(tau_max),tau_mean,np.exp(sigma_tau))) # -norm.cdf(np.log(tau_min),tau_mean,np.exp(sigma_tau))
    #        )*np.exp(sigma_tau)+tau_mean)
    
    ############################################################################################################
    ############################################################################################################
    
    # Car-following model specification

    # BE CAREFUL!! Below we add a small correction term (np.exp(-50)) to speed and relative speed.
    # This correction is included to avoid estimation issues if the value of the independent variables is 0.
    
    DSpace = beta1 + beta2*lag_speed
    
    fi = alpha1*Lagged_relative_speed + alpha2 * (Space_headway - DSpace)

    # Density functions for acceleration and deceleration
    P = norm.pdf(Acceleration,fi,np.exp(sigma))
    
    # Combined probability of acceleration and deceleration regimes
#     P = pdf_acc*(Speed_lead - Speed>=0)+pdf_dec*(Speed_lead - Speed<0)
    
    # We must include the density function of reaction time
    # This is ONLY needed for numerical integration or importance sampling (not for simulation)
    pdf_tau = ((1/(np.exp(sigma_tau)*tau))*norm.pdf((np.log(tau)-tau_mean)/np.exp(sigma_tau))/
               (norm.cdf(np.log(tau_max),tau_mean,np.exp(sigma_tau)))) # -norm.cdf(np.log(tau_min),tau_mean,(sigma_tau)))
    
    # Sampling distribution
    # This is ONLY needed for importance sampling
    pdf_tau0 = ((1/(np.exp(sigma_tau0)*tau))*norm.pdf((np.log(tau)-tau_mean0)/np.exp(sigma_tau0))/
                (norm.cdf(np.log(tau_max),tau_mean0,np.exp(sigma_tau0)))) # -norm.cdf(np.log(tau_min),tau_mean,(sigma_tau))
    
    # We must define the weights
    weight = (running_task==1)*pdf_tau/pdf_tau0*1+(running_task>1)*1
    
    # The total probability is
    P = P*weight
    
    ############################################################################################################
    # - Now this below is relevant if we have panel data and apply mixing (Do not change this piece of code!) -#
    if panel == 1:
    # Do it as panel
        P = pd.DataFrame(P)
        P = pd.concat([pd.Series(ID), pd.DataFrame(P)], axis=1, ignore_index=True)
        P.rename(columns={P.columns[0]: 'ID'},inplace=True)
    
        # We take the product of probabilities per individual per draw and then delete the ID column
        P = P.groupby('ID', as_index=False).prod()
        P = P.drop('ID', axis=1)
   
    if mixing == 1:
        # We take the average per row to get the average probability per individual (if mixing == 1)
        
        if panel == 1:
            P['mean'] = P.mean(axis=1)
            P = np.array(P['mean'])
        
        if panel == 0:
            P = pd.DataFrame(P)
            P = pd.concat([pd.Series(ID), pd.DataFrame(P)], axis=1, ignore_index=True)
            P.rename(columns={P.columns[0]: 'ID'},inplace=True)
    
            # We take the product of probabilities per individual per draw and then delete the ID column
            P = P.groupby('ID', as_index=False).prod()
            P = P.drop('ID', axis=1)
            P['mean'] = P.mean(axis=1)
            P = np.array(P['mean'])
            
    P = np.array(P)
    ### --- This is where the panel data approach ends. --- ###
    ############################################################################################################
    
    # We then take the log of the density function
    logprob = np.log(P)
    
    return logprob

### Function 2: sum of log likelihood (SLL)
This function simply takes the sum of log likelihood that we calculated with the first function

In [20]:
def SLL(betas):
    return -sum(LL(betas))

## Model estimation

### Warnings

Sometimes, optimisation procedures may 'overdo' it with warnings during estimation.
We can supress these with the piece of code below (not always advisable)

In [21]:
import warnings
from scipy.stats import t
# with warnings.catch_warnings():
warnings.filterwarnings("ignore")

### Estimation

Now it is finally time to run our estimation command.
We use an optimisation algorith called 'BFGS'.

**It is advisable not to edit the lines of code below**

In [22]:
# This will give us the initial loglikelihood value as an output
def callback1(betas):
    print("Current log likelihood:", -SLL(betas))

# This function will allow as to store parameter estimates during iterations
# Initialise list to store parameter values
parameter_values = [np.array(list(betas_start.values()))]
# Then define the function
def callback2(betas):    
    parameter_values.append(betas)
    column_names = list(betas_start.keys())
    with open(f'{model_name}_iterations.csv','w',newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(column_names)
        writer.writerows(parameter_values)

# Now let's combine the two callback functions
def combined_callback(betas):
    callback1(betas)
    callback2(betas)
        
print("Initial log likelihood:", -SLL(np.array(list(betas_start.values()))))

# Choose optimisation routine (preferably BFGS)
optimiser = 'BFGS' # BFGS or L-BFGS-B or nelder-mead

result = minimize(SLL, np.array(list(betas_start.values())), method=optimiser,callback=combined_callback, 
                  options={'disp':False}) # ,bounds=bounds1
#args = (parameter_values,)
print("Final log likelihood:", -result.fun)

Initial log likelihood: -24996.769451356184
Current log likelihood: -24943.85304157493
Current log likelihood: -22993.266882071603
Current log likelihood: -21985.946149958432
Current log likelihood: -21711.483248224344
Current log likelihood: -21709.54420738304
Current log likelihood: -21691.348557709818
Current log likelihood: -21686.262828863524
Current log likelihood: -21678.173350628203
Current log likelihood: -21674.298008512138
Current log likelihood: -21674.057639214097
Current log likelihood: -21673.890270579035
Current log likelihood: -21672.402337906286
Current log likelihood: -21669.473395580044
Current log likelihood: -21667.65027084301
Current log likelihood: -21665.751095677402
Current log likelihood: -21663.02748401397
Current log likelihood: -21660.87177828853
Current log likelihood: -21657.750065896875
Current log likelihood: -21652.83768110296
Current log likelihood: -21648.76021488762
Current log likelihood: -21647.854444112854
Current log likelihood: -21646.67486445

## Post estimation processing

We evaluate our parameter estimates using t-ratios (or p-Values).

In maximum likelihood estimation, we extract these from the variance-covariance matrix of the parameters.

The variance covariance matrix is not readily available but we need to calculate it.

This is done with the code below.

**DO NOT EDIT THE CHUNK OF CODE BELOW!**

In [ ]:
# Vector of parameter estimates
parameters = result['x'] 

# Calculate hessian
print("Calculating Hessian, please wait (this may take a while...)")
Hess = nd.Hessian(SLL,method = 'forward')
hessian = Hess(parameters)
inv_hessian = np.linalg.inv(hessian)

# Parameter statistics
dof = Nobs - len(betas_start) - 1
se = np.sqrt(np.diag(inv_hessian)) # Standard errors
tratio = parameters/se # t-ratios
p_value = (1-t.cdf(np.abs(tratio),dof)) * 2 # p-values


# --- Sandwich estimator --- #

# The sandwich estimator provides the "robust" s.e., t-ratios and p-values.
# These should be preferred over the classical parameter statistics.

# We first need the gradients at the solution point
Grad = nd.Gradient(LL,method = 'forward')
gradient = Grad(parameters)

# Then we need to calculate the B matrix
B = np.array([])
for r in range(gradient.shape[0]):
    Bm = np.zeros([len(betas_start),len(betas_start)])
    gradient0 = gradient[r,:]
    for i in range(len(gradient0)):
            for j in range(len(gradient0)):
                element = gradient0[i]*gradient0[j]
                Bm[i][j] = element
    if B.size==0:
                    B = Bm
    else:
                    B = B+Bm

# Finally we "sandwich" the B matrix between the inverese hessian matrices
BHHH = (inv_hessian)@(B)@(inv_hessian)

print("... Done!!")

# Now it is time to calculate some "robust" parameter statistics
rob_se = np.sqrt(np.diag(BHHH)) # robust standard error
rob_tratio = parameters/rob_se # robust t-ratio
rob_p_value = (1-t.cdf(np.abs(rob_tratio),dof)) * 2 # robust p-value

Calculating Hessian, please wait (this may take a while...)


## Results
Finally, we got our results. Let's print them!

The outputs that we receive are:
* Estimates: These are the values of our parameters. We must check if the sign is consistent with our expectations
* s.e.: Standard errors of the parameters
* tratio: t-ratio of the parameters (significant if absolute value > 1.96)
* p_value: p-value of the parameters (significant if < 0.05)

The parameter statistics also have their **robust** versions. These should be preferred as they are less susceptible to mis-specified models.

In [ ]:
arrays = np.column_stack((np.array(list(betas_start.keys())),parameters,se,tratio,p_value,rob_se,rob_tratio,rob_p_value))
results = pd.DataFrame(arrays, columns = ['Parameter','Estimate','s.e.','t-ratio0','p-value',
                                          'Rob s.e.','Rob t-ratio0','Rob p-value'])

results[['Estimate','s.e.','t-ratio0','p-value','Rob s.e.','Rob t-ratio0','Rob p-value']] = (
results[['Estimate','s.e.','t-ratio0','p-value','Rob s.e.','Rob t-ratio0','Rob p-value']].apply(pd.to_numeric, errors='coerce'))
numeric_cols = results.select_dtypes(include='number').columns
results[numeric_cols] = results[numeric_cols].round(3)
results # print results

## Goodness-of-fit indices

Let's calculate some goodness-of-fit indices now (do not edit)

In [ ]:
# First let's calculate the GoF indices

rho_squared = 1 - ((-result.fun)/(-SLL(np.zeros(len(betas_start)))))
adj_rho_squared = 1 - (((-result.fun)-len(betas_start))/(-SLL(np.zeros(len(betas_start)))))

AIC = 2*len(betas_start) - 2*(-result.fun)
BIC = len(betas_start)*np.log(Nobs) - 2*(-result.fun)

LL0t = "Log likelihood at zeros:" + str(-SLL(np.zeros(len(betas_start))))
LLinit = "Initial log likelihood:" + str(-SLL(np.array(list(betas_start.values()))))
LLfin = "Final log likelihood:" + str(-result.fun)

rs1 = "rho squared="+str(rho_squared)
rs2 = "adjusted rho squared="+str(adj_rho_squared)
ac = "AIC="+str(AIC)
bc = "BIC="+str(BIC)

# Then let's print the GoF indices

print(LL0t)
print(LLinit)
print(LLfin)

print(rs1)
print(rs2)
print(ac)
print(bc)

## Save output

We can save our output using the code below (do not edit):

In [ ]:
with open(f"{model_name}_results.txt", 'w') as f:
    f.write(f'{LL0t}\n')
    f.write(f'{LLinit}\n')
    f.write(f'{LLfin}\n')
    f.write(f'{rs1}\n')
    f.write(f'{rs2}\n')
    f.write(f'{ac}\n')
    f.write(f'{bc}\n')
    results.to_csv(f, index=False, sep='\t')
results.to_csv(f'{model_name}_results.csv', index=False)